In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def add_advanced_features(df):
    # (1) 공격 생산성 (Productivity): 득점 상황에서의 종합적인 기여도
    df['Finishing_Power'] = df['Finishing'] * 0.6 + df['ShotPower'] * 0.4
    df['Offensive_IQ'] = df[['Positioning', 'Vision', 'Reactions']].mean(axis=1)
    
    # (2) 신체적 완성도 (Physicality): 나이 대비 힘과 속도의 조화
    df['Physical_Potential'] = df[['Strength', 'Jumping', 'Stamina', 'SprintSpeed']].mean(axis=1)
    
    # (3) 기술적 잠재력 (Technical Upside): 드리블과 볼 컨트롤
    df['On_the_Ball'] = df[['Dribbling', 'BallControl', 'Agility', 'Balance']].mean(axis=1)
    
    # (4) 심리적 성숙도 (Clutch Factor): 압박 상황에서의 침착함
    df['Clutch_Score'] = df[['Composure', 'Penalties', 'Volleys']].mean(axis=1)
    
    # [핵심] 종합 잠재력 지표 (나이가 어릴수록 가산점)
    # 스탯들의 총합을 나이로 나누어 '떡잎' 점수 산출
    total_skill = df['Finishing_Power'] + df['Offensive_IQ'] + df['On_the_Ball'] + df['Physical_Potential']
    df['Prospect_Index'] = (total_skill * df['Clutch_Score']) / (df['Age'] + 1)
    
    return df

# 전처리 적용
train_adv = add_advanced_features(train)
test_adv = add_advanced_features(test)

# 2. 학습 설정
# 문자열 및 타겟 제외한 모든 수치형 데이터 사용
features = train_adv.select_dtypes(include=[np.number]).columns.drop(['Prospect']).tolist()
X_train = train_adv[features]
y_train = train_adv['Prospect']
X_test = test_adv[features]

# 3. 모델 학습 (파라미터 최적화)
model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.03,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)
model.fit(X_train, y_train)

# 4. 확률 예측
test_adv['Prospect_Prob'] = model.predict_proba(X_test)[:, 1]

# 5. 공격수 포지션 필터링 및 TOP 10 선정
fw_positions = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF']
final_list = test_adv[test_adv['Position'].isin(fw_positions)].sort_values(
    by='Prospect_Prob', ascending=False
).head(10)

# 6. 결과 출력 (다양한 지표 포함)
print("\n" + "="*85)
print("🏆차세대 공격수 유망주 TOP 10")
print("-" * 85)
output_cols = ['ID', 'Age', 'Position', 'Finishing', 'STRating', 'Offensive_IQ', 'Physical_Potential', 'Prospect_Prob']
print(final_list[output_cols].to_string(index=False))
print("="*85)


🏆차세대 공격수 유망주 TOP 10
-------------------------------------------------------------------------------------
       ID  Age Position  Finishing  STRating  Offensive_IQ  Physical_Potential  Prospect_Prob
TEST_1548   17       ST     5.9341    5.2055      4.566433            5.946325       0.992952
TEST_1224   16       ST     5.7143    5.2055      3.543667            5.240875       0.992912
TEST_1066   16       ST     7.1429    5.8904      5.181267            3.765575       0.989161
TEST_0191   18       ST     6.0440    5.4795      4.601900            5.463100       0.985901
TEST_0930   17       ST     5.6044    5.0685      4.103400            6.316525       0.984000
TEST_0342   18       ST     6.1538    5.2055      4.513900            5.581350       0.983743
TEST_0139   17       ST     6.0440    4.9315      4.251433            5.143750       0.983161
TEST_1327   18       ST     5.7143    5.2055      4.243267            5.227075       0.982973
TEST_0322   17       ST     6.1538    5.3425   